In [1]:
# 1. Imports & data loading 
import numpy as np
import plotly.graph_objects as go
import ipywidgets as wd
from vedo import load   # light wrapper around vtkXMLImageDataReader
from IPython.display import display

# --- Load VTI volume ---
VTI_FILE = "mixture.vti"  
volume   = load(VTI_FILE)

# scalar field (1-D array) & grid coordinates
values = np.asarray(volume.pointdata[0]).astype(float)
nx, ny, nz = volume.dimensions()

x, y, z = np.meshgrid(np.arange(nx),
                      np.arange(ny),
                      np.arange(nz),
                      indexing='ij')

# Flatten arrays – Plotly expects 1-D lists
xf = x.flatten(); yf = y.flatten(); zf = z.flatten()

data_min, data_max = values.min(), values.max()
print(f"Loaded grid {nx}×{ny}×{nz}, value-range = [{data_min:.2f}, {data_max:.2f}]")

#2. FigureWidgets (Isosurface + Histogram) 

# A. Isosurface figure
iso_fig = go.FigureWidget(
    data=[go.Isosurface(
        x=xf, y=yf, z=zf, value=values,
        isomin=0.0, isomax=0.0 + 1e-3,
        surface_count=1,
        caps=dict(x_show=False, y_show=False, z_show=False),
        colorscale='Plasma'
    )],
    layout=dict(title="Isosurface", margin=dict(l=0,r=0,b=0,t=30))
)

# B. Histogram figure
hist_fig = go.FigureWidget(
    data=[go.Histogram(
        x=values,
        nbinsx=60,
        marker=dict(color='lightblue', line=dict(color='black', width=1))  # Bin visibility
    )],
    layout=dict(title="Value Histogram",
                xaxis_title="Scalar value",
                yaxis_title="Count")
)

# 3. Slider & Reset button (ipywidgets) 

step = (data_max - data_min) / 200

slider = wd.FloatSlider(
    description='Isovalue',
    value=0.0, min=data_min, max=data_max,
    step=step, continuous_update=False,
    layout=wd.Layout(width='95%')
)

reset_btn = wd.Button(
    description='Reset',
    button_style='info',
    layout=wd.Layout(width='10%', margin='0px 10px')
)

# 4. Callback logic 

def update_plots(change):
    iso = change['new']
    band = 0.25

    # --- update isosurface trace
    iso_trace = iso_fig.data[0]
    iso_trace.isomin = iso - 1e-3
    iso_trace.isomax = iso + 1e-3

    # --- update histogram
    mask = (values >= iso - band) & (values <= iso + band)
    hist_fig.data[0].x = values[mask] if mask.any() else []
    hist_fig.layout.xaxis.title = f"Values in [{iso-band:.2f}, {iso+band:.2f}]"

slider.observe(update_plots, names='value')

def reset(_):
    slider.value = 0.0  # triggers update_plots
reset_btn.on_click(reset)

# 5. Display layout 

controls = wd.HBox([slider, reset_btn])  # horizontally aligned below
layout = wd.VBox([
    wd.HBox([iso_fig, hist_fig]),          # side-by-side plots
    controls                                # full-width controls below
])

display(layout)


Loaded grid 75×75×75, value-range = [-0.99, 0.43]


    'data': [{'caps': {'x': {'show': False}, 'y': {'show': False}…